In [3]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
pd.set_option("display.max_rows", None, "display.max_columns",
              None, 'display.max_colwidth', None)

from matplotlib import pyplot as plt
import numpy as np
import os


In [28]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [4]:
path_dataset = os.path.join(os.path.dirname(os.getcwd()), 'Dataset')

path_result = os.path.join(os.path.dirname(os.getcwd()), 'Result')
if not os.path.exists(path_result):
   os.makedirs(path_result)

path_information = os.path.join(path_result, 'Information')
if not os.path.exists(path_information):
   os.makedirs(path_information)
   
path_challenge = os.path.join(path_result, 'Challenge')
if not os.path.exists(path_challenge):
   os.makedirs(path_challenge)
   
path_solution = os.path.join(path_result, 'Solution')
if not os.path.exists(path_solution):
   os.makedirs(path_solution)

path_information_general = os.path.join(path_information, 'General')
if not os.path.exists(path_information_general):
   os.makedirs(path_information_general)

path_challenge_evolution = os.path.join(path_challenge, 'Evolution')
if not os.path.exists(path_challenge_evolution):
   os.makedirs(path_challenge_evolution)

path_solution_evolution = os.path.join(path_solution, 'Evolution')
if not os.path.exists(path_solution_evolution):
   os.makedirs(path_solution_evolution)

In [8]:
# combine issues and questions

df_issues = pd.read_json(os.path.join(path_dataset, 'issues.json'))
df_questions = pd.read_json(os.path.join(path_dataset, 'questions.json'))

df_issues['Challenge_link'] = df_issues['Issue_link']
df_issues['Challenge_original_content'] = df_issues['Issue_original_content']
df_issues['Challenge_preprocessed_content'] = df_issues['Issue_preprocessed_content']
df_issues['Challenge_summary'] = df_issues['Issue_gpt_summary']
df_issues['Challenge_creation_time'] = df_issues['Issue_creation_time']
df_issues['Challenge_comment_count'] = df_issues['Issue_comment_count']
df_issues['Challenge_score'] = df_issues['Issue_upvote_count'] - df_issues['Issue_downvote_count']
df_issues['Challenge_closed_time'] = df_issues['Issue_closed_time']
df_issues['Solution_summary'] = df_issues['Fix_manual_summary']

df_questions['Challenge_link'] = df_questions['Question_link']
df_questions['Challenge_original_content'] = df_questions['Question_original_content']
df_questions['Challenge_preprocessed_content'] = df_questions['Question_preprocessed_content']
df_questions['Challenge_summary'] = df_questions['Question_gpt_summary']
df_questions['Challenge_creation_time'] = df_questions['Question_creation_time']
df_questions['Challenge_comment_count'] = df_questions['Question_comment_count']
df_questions['Challenge_score'] = df_questions['Question_score']
df_questions['Challenge_closed_time'] = df_questions['Question_closed_time']
df_questions['Solution_original_content'] = df_questions['Answer_original_content']
df_questions['Solution_preprocessed_content'] = df_questions['Answer_preprocessed_content']
df_questions['Solution_summary'] = df_questions['Answer_gpt_summary']

del df_issues['Issue_title']
del df_issues['Issue_body']
del df_issues['Issue_link']
del df_issues['Issue_creation_time']
del df_issues['Issue_comment_count']
del df_issues['Issue_upvote_count']
del df_issues['Issue_downvote_count']
del df_issues['Issue_original_content']
del df_issues['Issue_preprocessed_content']
del df_issues['Issue_gpt_summary_original']
del df_issues['Issue_gpt_summary']
del df_issues['Issue_closed_time']
del df_issues['Fix_manual_summary_original']
del df_issues['Fix_manual_summary']

del df_questions['Question_title']
del df_questions['Question_body']
del df_questions['Question_link']
del df_questions['Question_creation_time']
del df_questions['Question_comment_count']
del df_questions['Question_score']
del df_questions['Question_original_content']
del df_questions['Question_preprocessed_content']
del df_questions['Question_gpt_summary_original']
del df_questions['Question_gpt_summary']
del df_questions['Question_closed_time']
del df_questions['Answer_list']
del df_questions['Answer_original_content']
del df_questions['Answer_preprocessed_content']
del df_questions['Answer_gpt_summary_original']
del df_questions['Answer_gpt_summary']

df_all = pd.concat([df_issues, df_questions], ignore_index=True)
df_all.to_json(os.path.join(path_dataset, 'original.json'),
               indent=4, orient='records')

In [9]:
# remove custom stop words from challenges and solutions

from gensim.parsing.preprocessing import remove_stopwords

# Refer to https://venturebeat.com/data-infrastructure/top-10-data-lake-solution-vendors-in-2022/
stop_words_custom = [
    'amazon',
    'answer',
    'appreci',
    'attempt',
    'aw',
    'azur',
    'challeng',
    'cloudera',
    'deep learn',
    'differ',
    'difficulti',
    'encount',
    'error',
    'experi',
    'databrick',
    'domo',
    'face',
    'fix',
    'gcp',
    'googl',
    'hei',
    'hello',
    'help',
    'ibm',
    'includ',
    'issu',
    'look',
    'machin learn',
    'microsoft',
    'need',
    'oracl',
    'possibl',
    'problem',
    'project',
    'provid',
    'python',
    'pytorch',
    'question',
    'resolv',
    'respond',
    'seek',
    'snowflak',
    'solut',
    'solv',
    'struggl',
    'suggest',
    'tensorflow',
    'thank',
    'try',
    'unabl',
    'us'
    'user'
]

df_all = pd.read_json(os.path.join(path_dataset, 'original.json'))

for index, row in df_all.iterrows():
    df_all.at[index, 'Challenge_original_content'] = remove_stopwords(row['Challenge_original_content'].replace(
        'Title: ', '').replace('Answer: ', ''), stopwords=stop_words_custom)
    df_all.at[index, 'Challenge_preprocessed_content'] = remove_stopwords(
        row['Challenge_preprocessed_content'].replace('Title: ', '').replace('Answer: ', ''), stopwords=stop_words_custom)
    df_all.at[index, 'Challenge_summary'] = remove_stopwords(row['Challenge_summary'], stopwords=stop_words_custom)

    if row['Solution_original_content']:
        df_all.at[index, 'Solution_original_content'] = remove_stopwords(row['Solution_original_content'].replace(
            'Title: ', '').replace('Answer: ', ''), stopwords=stop_words_custom)
        df_all.at[index, 'Solution_preprocessed_content'] = remove_stopwords(
            row['Solution_preprocessed_content'].replace('Title: ', '').replace('Answer: ', ''), stopwords=stop_words_custom)
        df_all.at[index, 'Solution_summary'] = remove_stopwords(row['Solution_summary'], stopwords=stop_words_custom)

df_all.to_json(os.path.join(path_dataset, 'preprocessed.json'),
               indent=4, orient='records')

In [10]:
df_all = pd.read_json(os.path.join(path_dataset, 'preprocessed.json'))

# remove issues with uninformed content
for index, row in df_all.iterrows():
    if len(row['Challenge_original_content'].split()) < 6 or len(row['Challenge_original_content']) < 30:
        print(row['Challenge_original_content'])
        df_all.drop(index, inplace=True)
    elif row['Solution_original_content'] and (len(row['Solution_original_content'].split()) < 6 or len(row['Solution_original_content']) < 30):
        print(row['Solution_original_content'])
        df_all.drop(index, inplace=True)

df_all.to_json(os.path.join(path_dataset, 'filtered.json'),
               indent=4, orient='records')

wonder initi studio lab titl
modulenotfounderror modul tensorboard
combin param param work
load
deploy fail
log val loss
import
tensorboard default logger option
logger
connect databas ye connect databas
run end run finish creat
vpc endpoint api api st
support hive adl
cloudform templat sure
csv file folder recordio
littl involv ye
file pin azurestor
processingstep uri code locat refer
leav case come accross
pipelin moment
cli sdk exampl
let know happen
delet creat endpoint
report appear bug team investig
code work
like templat run creat instanc
passau refer visual studio magazin
bug got correct todai close
upgrad sqlalchemi
file path ensur correct path
tri work let file transient
think chang log stream servic
situat chang git avail
sdk support automl sdk support
actual come smdebug version downgrad
updat core
forc gpu devic devic gpu
begin list
accord document maximum
sure estim framework version version
web termin import delet
ye maxtrialscallback exact featur situat
add run flush en

In [11]:
df_challenge = pd.read_json(os.path.join(path_dataset, 'topics.json'))
df_challenge['Challenge_solved_time'] = df_challenge['Challenge_closed_time'] - df_challenge['Challenge_creation_time']

df_topics = []

for name, group in df_challenge.groupby('Challenge_topic'):
    Mean_score = group['Challenge_score'].mean()
    Mean_comment_count = group['Challenge_comment_count'].mean()
    Score_comment_ratio = (Mean_score / Mean_comment_count).round(2)
    Count = group['Challenge_topic'].count()
    Solved_count = group['Challenge_closed_time'].notna().sum()
    Solved_ratio = (Solved_count / Count).round(2)
    Mean_solved_time = group['Challenge_solved_time'].mean(skipna=True) / pd.Timedelta(hours=1)
    Median_solved_time = group['Challenge_solved_time'].median(skipna=True) / pd.Timedelta(hours=1)
    topic_info = {
        'Challenge_topic': name,
        'Mean_score': Mean_score,
        'Mean_comment_count': Mean_comment_count,
        'Score_comment_ratio': Score_comment_ratio,
        'Count': Count,
        'Solved_count': Solved_count,
        'Solved_ratio': Solved_ratio,
        'Mean_solved_time': Mean_solved_time,
        'Median_solved_time': Median_solved_time,
    }
    df_topics.append(topic_info)

df_topics = pd.DataFrame(df_topics)
df_topics.to_json(os.path.join(path_information_general, 'general.json'), indent=4, orient='records')
df_topics = df_topics.set_index('Challenge_topic')

fig = df_topics.sort_values('Mean_score', ascending=False)['Mean_score'].plot(kind='bar', figsize=(15, 8), title='Challenge mean score', rot=15).get_figure()
fig.savefig(os.path.join(path_information_general, 'mean_score.png'))
plt.close()

fig = df_topics.sort_values('Score_comment_ratio', ascending=False)['Score_comment_ratio'].plot(kind='bar', figsize=(15, 8), title='Challenge score comment ratio', rot=15).get_figure()
fig.savefig(os.path.join(path_information_general, 'score_comment_ratio.png'))
plt.close()

fig = df_topics.sort_values('Solved_ratio')['Solved_ratio'].plot(kind='bar', figsize=(15, 8), title='Challenge Solved ratio', rot=15).get_figure()
fig.savefig(os.path.join(path_information_general, 'solved_ratio.png'))
plt.close()

fig = df_topics.sort_values('Mean_solved_time', ascending=False)['Mean_solved_time'].plot(kind='bar', figsize=(15, 8), title='Challenge median solved time', rot=15).get_figure()
fig.savefig(os.path.join(path_information_general, 'mean_solved_time.png'))
plt.close()

fig = df_topics.sort_values('Median_solved_time', ascending=False)['Median_solved_time'].plot(kind='bar', figsize=(15, 8), title='Challenge mean solved time', rot=15).get_figure()
fig.savefig(os.path.join(path_information_general, 'median_solved_time.png'))
plt.close()


In [4]:
import scipy.interpolate
from statsmodels.nonparametric.smoothers_lowess import lowess as sm_lowess

def smooth(x, y, xgrid, lowess_kw=None):
    samples = np.random.choice(len(x), 50, replace=True)
    y_s = y[samples]
    x_s = x[samples]
    y_sm = sm_lowess(y_s, x_s, **lowess_kw)
    # regularly sample it onto the grid
    y_grid = scipy.interpolate.interp1d(x_s, y_sm, fill_value='extrapolate')(xgrid)
    return y_grid

def lowess_with_confidence_bounds(x, y, conf_interval=0.95, lowess_kw=None):
    """
    Perform Lowess regression and determine a confidence interval by bootstrap resampling
    """
    xgrid = np.linspace(x.min(),x.max())
    
    K = 100
    smooths = np.stack([smooth(x, y, xgrid, lowess_kw) for _ in range(K)]).T

    mean = np.nanmean(smooths, axis=1)
    stderr = scipy.stats.sem(smooths, axis=1)
    
    clower = np.nanpercentile(smooths, (1-conf_interval)*50, axis=1)
    cupper = np.nanpercentile(smooths, (1+conf_interval)*50, axis=1)
    
    return xgrid, mean, stderr, clower, cupper

In [73]:
df_all = pd.read_json(os.path.join(path_dataset, 'topics.json'))
# BigQuery Stack Overflow public dataset is updated until Nov 24, 2022, 1:39:22 PM UTC-5
min(df_all['Challenge_creation_time']), max(df_all['Challenge_creation_time'])

(Timestamp('2014-09-14 22:12:24.493000'),
 Timestamp('2023-02-21 18:36:06.284000'))

In [ ]:
# Explore challenge topics evolution

df_challenge = pd.read_json(os.path.join(path_dataset, 'topics.json'))
df_challenge = df_challenge[(df_challenge['Challenge_creation_time'] > '2014-09-14') & (df_challenge['Challenge_creation_time'] < '2022-11-21')]

for name, group in df_challenge.groupby('Challenge_topic'):
    group = group.groupby(pd.Grouper(key='Challenge_creation_time', freq='2W')).agg(Count = ('Challenge_topic', 'count')).reset_index()
    x=pd.to_datetime(group['Challenge_creation_time']).values
    x=np.array([i.astype('datetime64[D]').astype(int) for i in x])
    y=group['Count'].values
    # 95% confidence interval
    xgrid, mean, stderr, clower, cupper = lowess_with_confidence_bounds(x, y, conf_interval=0.95, lowess_kw={"frac": 0.5, "it": 5, "return_sorted": False})
    x=pd.to_datetime(group['Challenge_creation_time']).values
    fig, ax = plt.subplots(figsize=(20,10))
    plt.plot(x, y, 'k.', label='Observations')
    plt.plot(xgrid, mean, color='tomato', label='LOWESS')
    plt.fill_between(xgrid, clower, cupper, alpha=0.3, label='LOWESS uncertainty')
    plt.legend(loc='best')
    fig.savefig(os.path.join(path_challenge_evolution, f'Topic_{name}'), bbox_inches="tight")
    plt.close()


In [28]:
df_all = pd.read_json(os.path.join(path_dataset, 'topics.json'))
df_solution = df_all[df_all['Solution_topic'] > -1]
# BigQuery Stack Overflow public dataset is updated until Nov 24, 2022, 1:39:22 PM UTC-5
min(df_solution['Challenge_creation_time']), max(df_solution['Challenge_creation_time'])

(Timestamp('2014-09-14 22:12:24.493000'),
 Timestamp('2023-02-21 18:36:06.284000'))

In [7]:
# Explore solution topics evolution

df_solution = pd.read_json(os.path.join(path_dataset, 'topics.json'))
df_solution = df_solution[df_solution['Solution_topic'] > -1]
df_solution = df_solution[(df_solution['Challenge_creation_time'] > '2014-09-14') & (df_solution['Challenge_creation_time'] < '2022-11-21')]

for name, group in df_solution.groupby('Solution_topic'):
    group = group.groupby(pd.Grouper(key='Challenge_closed_time', freq='W')).agg(Count = ('Solution_topic', 'count')).reset_index()
    x=pd.to_datetime(group['Challenge_closed_time']).values
    x=np.array([i.astype('datetime64[D]').astype(int) for i in x])
    y=group['Count'].values
    # 95% confidence interval
    xgrid, mean, stderr, clower, cupper = lowess_with_confidence_bounds(x, y, conf_interval=0.95, lowess_kw={"frac": 0.5, "it": 5, "return_sorted": False})
    x=pd.to_datetime(group['Challenge_closed_time']).values
    fig, ax = plt.subplots(figsize=(20,10))
    plt.plot(x, y, 'k.', label='Observations')
    plt.plot(xgrid, mean, color='tomato', label='LOWESS')
    plt.fill_between(xgrid, clower, cupper, alpha=0.3, label='LOWESS uncertainty')
    plt.legend(loc='best')
    fig.savefig(os.path.join(path_solution_evolution, f'Topic_{name}'), bbox_inches="tight")
    plt.close()


In [13]:
# average solved time
df_all = pd.read_json(os.path.join(path_dataset, 'topics.json'))
df_all['Challenge_solved_time'] = df_all['Challenge_closed_time'] - df_all['Challenge_creation_time']
df_solved = df_all[~df_all['Challenge_solved_time'].isna()]

for name, group in df_solution.groupby('Challenge_topic'):
    
    

# df_solved['Challenge_solved_time']

0                173 days 18:43:03
2                 55 days 09:04:07
3                  0 days 05:52:43
4                198 days 18:03:20
5                 16 days 23:33:26
6                 84 days 23:49:06
7                184 days 01:57:43
9                 10 days 01:13:34
10                 0 days 00:08:32
11                 0 days 22:31:45
12                 2 days 04:36:40
13                13 days 08:57:11
14                88 days 12:30:07
15               265 days 05:12:24
16                 0 days 03:47:20
17                 5 days 19:21:14
18                 0 days 01:11:07
21                32 days 22:23:28
22                27 days 05:47:41
23                 2 days 21:54:01
24                 3 days 23:05:15
25                11 days 14:18:06
26                39 days 09:50:27
27               181 days 04:25:47
30                 5 days 02:45:59
31                42 days 09:57:17
34                39 days 11:20:27
35               409 days 06:23:49
36                 6